In [2]:
import numpy as np 
import gym
import pickle, os

## Problem Statement:
There are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger’s location, picks up the passenger, drives to the passenger’s destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends.

Actions
There are 6 discrete deterministic actions:

0: move south

1: move north

2: move east

3: move west

4: pickup passenger

5: drop off passenger

Observations
There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is in the taxi), and 4 destination locations.

Note that there are 400 states that can actually be reached during an episode. The missing states correspond to situations in which the passenger is at the same location as their destination, as this typically signals the end of an episode. Four additional states can be observed right after a successful episodes, when both the passenger and the taxi are at the destination. This gives a total of 404 reachable discrete states.

Each state space is represented by the tuple: (taxi_row, taxi_col, passenger_location, destination)

An observation is an integer that encodes the corresponding state. The state tuple can then be decoded with the “decode” method.

Passenger locations:

0: R(ed)

1: G(reen)

2: Y(ellow)

3: B(lue)

4: in taxi

Destinations:

0: R(ed)

1: G(reen)

2: Y(ellow)

3: B(lue)

#### Source : https://www.gymlibrary.dev/environments/toy_text/taxi/

In [3]:
env = gym.make("Taxi-v3", render_mode="human").env

env.reset()
env.render()

print("Action Space {}".format(env.action_space))

print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


## Problem and Solution

The SARSA algorithm is a reinforcement learning (RL) algorithm that is often used to solve the Taxi problem. The Taxi problem is a classic RL problem where an agent (a taxi) needs to navigate a gridworld to pick up passengers and drop them off at their specified locations. The goal of the agent is to learn an optimal policy that maximizes its total reward.

Here's an overview of how the SARSA algorithm works:

State and Action Representation:

The gridworld is divided into discrete states, representing the different configurations of the taxi, passengers, and destinations.
Actions represent the possible moves the taxi can make: move north, south, east, or west, or pick up or drop off a passenger.
Q-Table Initialization:

The algorithm initializes a Q-table, which is a table that maps state-action pairs to their corresponding values.
The Q-table is initially set to small random values or zeros.
Exploration vs. Exploitation:

The agent follows an exploration-exploitation strategy to balance between exploring new actions and exploiting the knowledge gained so far.
During exploration, the agent takes random actions with a certain probability (epsilon-greedy exploration).
During exploitation, the agent selects actions based on the current estimates in the Q-table.
Action Selection:

Based on the current state, the agent selects an action using the epsilon-greedy strategy.
With probability (1-epsilon), the agent selects the action with the highest Q-value for that state (exploitation).
With probability epsilon, the agent selects a random action (exploration).
Action Execution and Reward:

The agent executes the selected action in the environment and observes the resulting state and the immediate reward.
The state transitions to a new state based on the action taken.
Update Q-Table:

The agent updates the Q-value of the previous state-action pair based on the observed reward and the Q-value of the new state-action pair.
The update rule in SARSA is as follows:
Q(s, a) = Q(s, a) + alpha * (r + gamma * Q(s', a') - Q(s, a))
Q(s, a) is the Q-value of state-action pair (s, a)
alpha is the learning rate, controlling the weight given to the new information (0 < alpha <= 1)
r is the observed reward for taking action a in state s
gamma is the discount factor, balancing the importance of immediate and future rewards (0 < gamma <= 1)
Q(s', a') is the Q-value of the new state-action pair after taking action a in state s
Repeat:

Steps 4 to 6 are repeated until the agent reaches the terminal state (a passenger is dropped off).

The agent updates the Q-values after each action taken, gradually improving its policy.
By repeatedly updating the Q-values based on the observed rewards and using the epsilon-greedy strategy to balance exploration and exploitation, the SARSA algorithm allows the agent to learn an optimal policy for navigating the Taxi problem.




In [4]:
env.P[123]

{0: [(1.0, 223, -1, False)],
 1: [(1.0, 23, -1, False)],
 2: [(1.0, 123, -1, False)],
 3: [(1.0, 103, -1, False)],
 4: [(1.0, 123, -10, False)],
 5: [(1.0, 123, -10, False)]}

In [5]:
# Defining parameters
epsilon = 1 # Total exploration and no exploitation
alpha = 0.3
gamma = 0.95

# Training parameters
max_episodes = 10  # number of episodes to use for training
max_steps = 100   # maximum number of steps per episode

#Initializing the Q-table 500x6
Q = np.zeros((env.observation_space.n, env.action_space.n))

In [8]:
# Function to choose the next action
def choose_action(state):

    action=0

    if np.random.uniform(0, 1) < epsilon:

        action = env.action_space.sample()   # explore

    else:
        
        action = np.argmax(Q[state, :])      # exploit
        
    return action

# Function to update the Q-value
def update(state, state2, reward, action, action2):
    
    predict = Q[state, action]
    target = reward + gamma * Q[state2, action2]
    
    Q[state, action] = Q[state, action] + alpha * (target - predict)

In [9]:
# Initializing parameters
score = 0
goal = 0 
episode = 0


# Starting the SARSA learning
for episode in range(max_episodes):

    # Print each episode
    print("Episode: ", episode)
    print("Goal reached: ", goal)
    print("Score: ", score)
    print(" ")
    
    state1 = env.reset()
    state1 = state1[0]
    action1 = choose_action(state1)
    
    # Reset parameters
    s = 0
    score = 0
    done = False
    
    # Decreasing epsilon
    epsilon -= 0.005
    
    if epsilon < 0.05:
        epsilon = 0.05    # epsilon min: total exploitation (95%)
 

    for s in range(max_steps):
        
        # Rendering
        env.render()
         
        # Get the next state
        state2, reward, done, truncate, info = env.step(action1)
 
        # Choose the next action
        action2 = choose_action(state2)
         
        # Update the Q-value
        update(state1, state2, reward, action1, action2)
 
        state1 = state2
        action1 = action2
         
        # Updating parameters
        s += 1
        score += reward
        
        if reward==20:
            goal +=1
         
        #If at the end of learning process
        if done:
            break

Episode:  0
Goal reached:  0
Score:  0
 
Episode:  1
Goal reached:  0
Score:  -406
 
Episode:  2
Goal reached:  0
Score:  -424
 
Episode:  3
Goal reached:  0
Score:  -334
 
Episode:  4
Goal reached:  0
Score:  -415
 
Episode:  5
Goal reached:  0
Score:  -370
 
Episode:  6
Goal reached:  0
Score:  -316
 
Episode:  7
Goal reached:  0
Score:  -334
 
Episode:  8
Goal reached:  0
Score:  -334
 
Episode:  9
Goal reached:  0
Score:  -316
 


: 

# Results

The program needs to run for a significant number of episodes, possibly ranging in the hundreds or thousands, which can take many hours to complete. Throughout the learning process, the agent initially focuses on exploration, leading to numerous mistakes as it explores various scenarios.

In the beginning, during the first episode, the agent engages in pure exploration, resulting in a failure to reach the goal and a negative score of -415.

After around 100 episodes, the balance between exploration and exploitation becomes equal, as the value of epsilon decreases. Although the agent is still learning, it can occasionally achieve some goals, although with a considerable negative score of -325.

Continuing further to about 200 episodes, the agent demonstrates a growing capability to reach goals more frequently, even though it still makes mistakes. At this point, the agent shifts into pure exploitation, having learned enough to make informed decisions. The score improves to -118.

Over the subsequent 100 episodes, from 201 to 300, the agent's success in reaching goals becomes more consistent, as evidenced by reaching the goal 92 times. The score further improves to -31.

After 400 episodes and beyond, the agent has gained extensive knowledge about how to interact optimally in each situation, consistently making the best decisions and taking the shortest paths to reach the goals. It becomes rare for the agent to fail in reaching the goal.

For example, in the 401st episode, the agent successfully reached the goal 182 times, resulting in a score of -2. As the number of episodes increases, the agent's performance continues to improve significantly, with 3779 successful goal completions out of 4000 episodes and a positive score of 10.